# FPUS23 Denoiser + YOLO Experiment (Colab)

This notebook trains a denoising autoencoder for ultrasound speckle, creates a despeckled training set, and compares YOLO mAP vs baseline.

In [ ]:
# Repo + pinned deps
!git clone https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git /content/fpus23
%cd /content/fpus23
!pip -q install -r requirements_colab.txt
!pip -q install gdown

In [ ]:
# Optional: save results to Google Drive
SAVE_TO_DRIVE = True
DRIVE_RESULTS = '/content/drive/MyDrive/FPUS23_runs'
if SAVE_TO_DRIVE:
    from google.colab import drive; drive.mount('/content/drive', force_remount=True)
    import os, pathlib as p
    p.Path(DRIVE_RESULTS).mkdir(parents=True, exist_ok=True)
    print('Results will be saved to', DRIVE_RESULTS)
else:
    print('Saving to local /content/fpus23_project/runs')

In [ ]:
# Download dataset
!gdown https://drive.google.com/uc?id=1LL-r2hNiP6C190UBSE4v1FFCF3OQT9N3 -O /content/FPUS23_Dataset.zip
!unzip -q /content/FPUS23_Dataset.zip -d /content/FPUS23_Dataset || true
!ls -lh /content/FPUS23_Dataset || true

In [ ]:
# Prepare dataset (XML -> YOLO -> COCO)
!python 'New folder/scripts/prepare_fpus23.py' \
  --dataset-root /content/FPUS23_Dataset/Dataset \
  --project-root /content/fpus23_project
!ls -lh /content/fpus23_project/dataset/fpus23_yolo/

In [ ]:
# Sanity-check (val split)
!python 'New folder/scripts/tools/verify_yolo_dataset.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --split val --limit 24

## Train denoiser (autoencoder)

In [ ]:
# Train denoiser on COCO train images
!python 'New folder/scripts/train_denoising_autoencoder.py' \
  --images-dir /content/fpus23_project/dataset/fpus23_coco/train \
  --epochs 20 \
  --batch-size 16 \
  --device cuda \
  --save-dir /content/fpus23_project/checkpoints/denoiser

## Create despeckled training set (optional)

In [ ]:
# Fast baseline: median blur offline
!python 'New folder/scripts/preprocess_fpus23_despeckle.py' \
  --input /content/fpus23_project/dataset/fpus23_yolo/images/train \
  --output /content/fpus23_project/dataset/fpus23_yolo_despeckled/images/train \
  --kernel 5 \
  --data-yaml /content/fpus23_project/dataset/fpus23_yolo/data.yaml
DESPECKLED_YAML = '/content/fpus23_project/dataset/fpus23_yolo_despeckled/data.yaml'
!sed -n '1,80p' $DESPECKLED_YAML

## Train YOLO: baseline vs despeckled
We compare yolo11s and yolo11m on original vs despeckled training sets.

In [ ]:
# Baseline on original
PROJECT_ARG = DRIVE_RESULTS if SAVE_TO_DRIVE else 'runs/detect'
!python 'New folder/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --model yolo11s.pt \
  --epochs 80 \
  --batch 16 \
  --imgsz 768 \
  --project $PROJECT_ARG \
  --name fpus23_denoiser_baseline_s
# On despeckled
!python 'New folder/scripts/train_yolo_fpus23_phase1.py' \
  --data $DESPECKLED_YAML \
  --model yolo11s.pt \
  --epochs 80 \
  --batch 16 \
  --imgsz 768 \
  --project $PROJECT_ARG \
  --name fpus23_denoiser_despeckle_s

In [ ]:
# Medium model (optional)
!python 'New folder/scripts/train_yolo_fpus23_phase1.py' \
  --data /content/fpus23_project/dataset/fpus23_yolo/data.yaml \
  --model yolo11m.pt \
  --epochs 80 \
  --batch 8 \
  --imgsz 768 \
  --project $PROJECT_ARG \
  --name fpus23_denoiser_baseline_m
!python 'New folder/scripts/train_yolo_fpus23_phase1.py' \
  --data $DESPECKLED_YAML \
  --model yolo11m.pt \
  --epochs 80 \
  --batch 8 \
  --imgsz 768 \
  --project $PROJECT_ARG \
  --name fpus23_denoiser_despeckle_m